In [1]:
from qiskit import *
from qiskit.visualization import *
from qiskit.circuit.library import MCMT

from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT

import numpy as np
import matplotlib.pyplot as plt
import math

sim = Aer.get_backend('qasm_simulator')

# using tikzplotlib to export plot to latex
import tikzplotlib

def tikzplotlib_fix_ncols(obj):
    """
    workaround for matplotlib 3.6 renamed legend's _ncol to _ncols, which breaks tikzplotlib
    """
    if hasattr(obj, "_ncols"):
        obj._ncol = obj._ncols
    for child in obj.get_children():
        tikzplotlib_fix_ncols(child)

def save_fig(name, width='10cm', height='10cm'):
    fig = plt.gcf()
    tikzplotlib_fix_ncols(fig)

    tikzplotlib.clean_figure()
    tikzplotlib.save(name + '.tex', axis_width=width, axis_height=height)

In [2]:
sudoku = [[0,0,0],
          [0,0,0],
          [0,0,0]]

In [3]:
register_input = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(len(sudoku)):
    for j in range(len(sudoku[i])):
        register_input[i][j] = QuantumRegister(2, 'q'+str(i)+str(j))
        
cond_bits = QuantumRegister(36, 'cond')
fin_cond = QuantumRegister(18, 'fin_cond')
out_cond = QuantumRegister(1, 'out_cond')
cr = ClassicalRegister(18, 'cr')

In [4]:
num_it = int(np.floor(np.pi / 4 * np.sqrt(2**18 / 21)))

In [5]:
qc = QuantumCircuit(*register_input[0], *register_input[1], *register_input[2], cond_bits, fin_cond, out_cond, cr)

for i in range(len(sudoku)):
    for j in range(len(sudoku[i])):
        if sudoku[i][j] == 0:
            qc.h(register_input[i][j])
        if sudoku[i][j] == 1:
            qc.x(register_input[i][j][0])    
        if sudoku[i][j] == 2:
            qc.x(register_input[i][j][1])
        if sudoku[i][j] == 3:
            qc.x(register_input[i][j][0:1])
            
qc.x(out_cond)
qc.h(out_cond)

qc.barrier()

def cond_verif():
    cond_act = 0
    for k in range(len(register_input)):
        qc.cx(register_input[k][0][0], cond_bits[cond_act])
        qc.cx(register_input[k][0][1], cond_bits[cond_act + 1])
        qc.cx(register_input[k][1][0], cond_bits[cond_act])
        qc.cx(register_input[k][1][1], cond_bits[cond_act + 1])
        cond_act += 2
        qc.cx(register_input[k][0][0], cond_bits[cond_act])
        qc.cx(register_input[k][0][1], cond_bits[cond_act + 1])
        qc.cx(register_input[k][2][0], cond_bits[cond_act])
        qc.cx(register_input[k][2][1], cond_bits[cond_act + 1])
        cond_act += 2
        qc.cx(register_input[k][1][0], cond_bits[cond_act])
        qc.cx(register_input[k][1][1], cond_bits[cond_act + 1])
        qc.cx(register_input[k][2][0], cond_bits[cond_act])
        qc.cx(register_input[k][2][1], cond_bits[cond_act + 1])
        cond_act += 2
    
    for k in range(len(register_input)):
        qc.cx(register_input[0][k][0], cond_bits[cond_act])
        qc.cx(register_input[0][k][1], cond_bits[cond_act + 1])
        qc.cx(register_input[1][k][0], cond_bits[cond_act])
        qc.cx(register_input[1][k][1], cond_bits[cond_act + 1])
        cond_act += 2
        qc.cx(register_input[0][k][0], cond_bits[cond_act])
        qc.cx(register_input[0][k][1], cond_bits[cond_act + 1])
        qc.cx(register_input[2][k][0], cond_bits[cond_act])
        qc.cx(register_input[2][k][1], cond_bits[cond_act + 1])
        cond_act += 2
        qc.cx(register_input[1][k][0], cond_bits[cond_act])
        qc.cx(register_input[1][k][1], cond_bits[cond_act + 1])
        qc.cx(register_input[2][k][0], cond_bits[cond_act])
        qc.cx(register_input[2][k][1], cond_bits[cond_act + 1])
        cond_act += 2
        
    
    for k in range(len(fin_cond)):
        qc.cx(cond_bits[2 * k], fin_cond[k])
        qc.cx(cond_bits[2 * k + 1], fin_cond[k])

def oracle():
    cond_verif()
    qc.mct(fin_cond, out_cond)
    cond_verif()
    
def diffuser():
    for k in range(len(sudoku)):
        for l in range(len(sudoku[k])):
            if sudoku[k][l] == 0:
                qc.h(register_input[k][l])
            if sudoku[k][l] == 1:
                qc.x(register_input[k][l][0])
            if sudoku[k][l] == 2:
                qc.x(register_input[k][l][1])
            if sudoku[k][l] == 3:
                qc.x(register_input[k][l][0:1])

            qc.x(register_input[k][l][0:1])
            
    qc.h(register_input[0][0][0])
    qc.mct([register_input[0][0][1], *register_input[0][1], *register_input[0][2], *register_input[1][0], *register_input[1][1], *register_input[1][2], *register_input[2][0], *register_input[2][1], *register_input[2][2]], register_input[0][0][0])
    qc.h(register_input[0][0][0])
    
    for k in range(len(sudoku)):
        for l in range(len(sudoku[k])):
            qc.x(register_input[k][l][0:1])
            
            if sudoku[k][l] == 0:
                qc.h(register_input[k][l])
            if sudoku[k][l] == 1:
                qc.x(register_input[k][l][0])
            if sudoku[k][l] == 2:
                qc.x(register_input[k][l][1])
            if sudoku[k][l] == 3:
                qc.x(register_input[k][l][0:1])      
 

for _ in range(num_it):
    oracle()
    diffuser()
    qc.barrier()
     
measure_i = 0
for i in range(len(register_input)):
    for j in range(len(register_input[i])):
        for h in range(2):
            qc.measure(register_input[i][j][h], cr[measure_i])
            measure_i += 1
            
print('circuit done')

circuit done


In [6]:
from qiskit import IBMQ

# IBMQ.save_account(TOKEN)
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers

provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

backend = provider.get_backend('simulator_mps')

C:\Users\romai\AppData\Local\Temp\ipykernel_10428\3157583564.py:4: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account() # Load account from disk
C:\Users\romai\AppData\Local\Temp\ipykernel_10428\3157583564.py:4: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account() # Load account from disk


In [ ]:
print('debut')
results = execute(qc, backend=backend, shots=4).result()
print('execute done')
counts = results.get_counts()
plot_histogram(counts)

debut


In [ ]:
with open('out.txt', 'w+') as f:
    print(counts, file=f)

plt.figure(figsize=(10, 10))
plt.barh(list(counts.keys()), list(counts.values()))
save_fig('sudoku_result')